# 🍈 **CB + CF로 추출된 유사곡을 기반으로 노래를 추천합니다.** 🎶
* song embedding을 위해 **content 자체의 정보**인 `meta정보(년,월,장르)`와 `태그` 정보에 더하여, **유저들이 어느 곡을 유사하다고 생각하였는지에 대한 정보**를 포함시키기 위하여 `play list`에 대한 feature를 추가하여 embedding하였다.
* **5개 layer를 가진 AE를 기반으로 d : 1024의 latent feature를 추출하였고** *cosine similarity를 이용하여 구한 유사곡 리스트를 기반으로 추천을 진행한다.

```
    Music nDCG: 0.107791
    Tag nDCG: 0.341254
    Score: 0.142811
```

In [1]:
import pickle

import pandas as pd
import numpy as np
from itertools import chain

# 🌳 **추천 전 몇가지 전처리**

### **index와 sid간 연결용 dictionary**

In [61]:
df_raw = one_hot_encoding = pd.read_pickle('/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/one_hot_encoding_nov_ply.pkl')

songs = df_raw.index.tolist()
index_to_sid = {i:songs[i] for i in range(len(songs))}
sid_to_index = {j:i for i,j in index_to_sid.items()}

items = df_raw.columns.tolist()
new_tags = items[:407]
# index_to_iid = {i:items[i] for i in range(len(items))}
# iid_to_index = {j:i for i,j in index_to_iid.items()}

song_meta = pd.read_pickle('/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/song_meta_nov.pkl')

del df_raw

### **sim_songs**

In [62]:
# with open('/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/nov/sim_songs_epoch30.pkl', 'rb') as f:
#       sim_songs_list = pickle.load(f)

with open('/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/nov/sim_songs_epoch30_cosim.pkl', 'rb') as f:
      sim_songs_list = pickle.load(f)

In [63]:
len(sim_songs_list)

226578

In [64]:
# 혹시 순서 뒤집혔는지 체크!
sim_songs_list[0][0] # 117563

117563

In [65]:
# index로 저장된 song들을 sid로 변환

for i in range(len(sim_songs_list)):
    sim_songs_list[i] = [index_to_sid[j] for j in sim_songs_list[i]]

In [66]:
# dictionary 생성
sim_songs = {index_to_sid[i]:sim_songs_list[i] for i in range(len(sim_songs_list))}

### **pop songs/tags**
* 전체 train set의 인기 태그인 `pop_tags`보다, 인기 곡의 인기 태그인 `pop_tags2`를 사용하는 경우에 tag 점수가 소폭 상승

In [67]:
tr = pd.read_json("/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/pickle/train.json", typ = 'frame')

In [68]:
from collections import Counter

pop_songs_counter = Counter(chain.from_iterable(tr.songs))
pop_tags_counter = Counter(chain.from_iterable(tr.tags))
pop_songs = [i for i,j in pop_songs_counter.most_common(100)]

# 전체에서 인기 태그를 선택하는 것 보다, 인기곡의 태그를 가져오는 것이 태그점수 소폭 상승
# pop_tags = [i for i,j in pop_tags_counter.most_common(10)]
# pop_tags

In [69]:
# 인기 곡의 인기 태그 추출

tag_counter = Counter(chain.from_iterable([sid_to_tag[sid] for sid in pop_songs]))
pop_tags2 = [tag for tag,count in tag_counter.most_common(10)]
pop_tags2

['발라드', '설렘', '사랑', '잔잔한', '감성', '기분전환', '비오는날', '카페', '드라이브', '밤']

### **tag_counter**
tag를 입력하면 가장 많이 매칭된 노래가 150곡씩 추출된다. (내림차순)

In [70]:
tr.loc[:,'new_tags'] = tr['tags'].map(lambda x: list(set(x)&set(new_tags)))

In [71]:
from collections import defaultdict

tag_to_counter = defaultdict(lambda : Counter())

In [72]:
for tags, songs in zip(tr.new_tags, tr.songs):
    for tag in tags:
        for sid in songs:
            tag_to_counter[tag][sid] += 1

In [73]:
tag_to_counter['발라드'].most_common(10)

[(680366, 545),
 (215411, 529),
 (463173, 514),
 (349492, 501),
 (427724, 501),
 (520093, 491),
 (235773, 484),
 (396828, 469),
 (442014, 443),
 (42155, 442)]

In [74]:
tag_to_sid = {tag:[i for i,j in counter.most_common(150)] for tag,counter in tag_to_counter.items()}

In [75]:
tag_to_sid['발라드'][:10]

[680366, 215411, 463173, 349492, 427724, 520093, 235773, 396828, 442014, 42155]

### **val dict**

In [76]:
val = pd.read_json("/content/drive/My Drive/Melon-PL-Continuation/0802/train_split/pickle/val_question.json", typ = 'frame')

In [77]:
val['new_tags'] = val.tags.map(lambda x: [i for i in x if i in new_tags])
val['new_songs'] = val.songs.map(lambda x: [i for i in x if i in sid_to_index.keys()])

In [78]:
val_dict = val[['id','new_songs','new_tags','songs','tags']].T.to_dict() # new songs의 유사곡에서 추천된 곡이 songs와 겹칠수는 없음.
val_dict = list(val_dict.values())

In [79]:
val_dict[0]

{'id': 147332,
 'new_songs': [],
 'new_tags': [],
 'songs': [],
 'tags': ['Billboard', 'uspop']}

### **sid_to_tag**

In [80]:
sid_to_tag = song_meta.set_index('id')['new_tags'].to_dict()

In [81]:
song_meta.query('id == 0')

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,"[비오는날, 드라이브, 추억, 회상]"


In [82]:
sid_to_tag[0]

['비오는날', '드라이브', '추억', '회상']

# 🌲 **추천에 사용할 함수 정의**

In [29]:
# 채워야할 100곡을 골고루 나눠준다.

def distribute(total):
    if total >= 100:
        return [1]*100

    default, remainder = divmod(100,total)
    cnt = [default]*total
    for i in range(total):
        if remainder == 0:
            break
        cnt[i] += 1
        remainder -= 1

    assert sum(cnt) == 100

    return cnt

In [44]:
# 유사곡을 기반으로 노래를 추천합니다.

def similarity_based_song_recommend(new_songs):
    rec_songs = []
    for song, count in zip(new_songs,distribute(len(new_songs))):
        temp_rec = 0
        for candidate in sim_songs[song]:
            if temp_rec == count:
                break
            if candidate not in songs and candidate not in rec_songs:
                rec_songs.append(candidate)
                temp_rec += 1

    return rec_songs

In [31]:
# tag를 기반으로 노래를 추천합니다.

def tag_based_song_recommend(new_tags, needed, total = 100):
    rec_songs = []
    for tag,count in zip(new_tags,distribute(len(new_tags))):
        for candidate in tag_to_sid[tag]:
            if len(rec_songs) == total:
                break
            if candidate not in songs and candidate not in rec_songs:
                rec_songs.append(candidate)
    return rec_songs

In [32]:
# 추천 곡을 받으면 이를 기반으로 tag를 추천해줍니다.

def tag_recommend(sid_list, seen_tag):
    assert len(set(sid_list)) == 100

    rec_tags = []

    # extract tags
    tag_counter = Counter(chain.from_iterable([sid_to_tag[sid] for sid in sid_list]))
    tag_candidates = [tag for tag,count in tag_counter.most_common(20)]

    for tag in tag_candidates:
        if tag not in seen_tag:
            rec_tags.append(tag)
            if len(rec_tags) == 10:
                break

    if len(rec_tags) != 10:
        fill = list(set(pop_tags2) - set(rec_tags))
        rec_tags.extend(fill[:10-len(rec_tags)])

    assert len(set(rec_tags)) == 10

    return rec_tags

In [33]:
# 추천 곡이 100곡보다 적다면, 인기곡을 통해 100곡을 채워줍니다.

def fill(rec_songs):
    if len(rec_songs) == 100:
        return rec_songs
    k = 100 - len(rec_songs)
    candidates = list(set(pop_songs) - set(rec_songs))
    return rec_songs + candidates[:k]

# 🧐 **결과 미리보기**

In [86]:
def recommendation_preview(index):
    id, new_songs, new_tags, songs, tags = val_dict[index].values()

    print("● Query")
    display(val[val.id == id])

    # 1 ) 참고할 노래가 없는 경우
    if len(new_songs) == 0:
         # 1. 노래는 없지만 tag가 있다면 tag를 기반으로 추천해준다.
        if new_tags:
            rec_songs = tag_based_song_recommend(new_tags, len(new_tags))
            rec_songs = fill(rec_songs) # 100곡을 채우지 못한 경우 인기곡으로
            rec_tags = tag_recommend(rec_songs, tags)
        else:
            # 2. 이용할 정보가 없는 경우 인기곡, 인기 태그를 추천한다.
            rec_songs = pop_songs
            rec_tags = pop_tags2

    # 2 ) 참고할 노래가 있는 경우 유사곡 k개(모두 골고루 가질 수 있도록 배치했음)씩을 모아 100곡을 완성
    else:
        rec_songs = similarity_based_song_recommend(new_songs)

        # tag가 없어 채우지 못한 경우 인기곡으로 채워준다.
        rec_songs = fill(rec_songs)
        rec_tags = tag_recommend(rec_songs, tags)

    print("\n● Recommended")
    print(rec_tags)
    display(song_meta.query(f'id in {rec_songs[:5]}'))
    display(song_meta.query(f'id in {rec_songs[-5:]}'))

In [87]:
recommendation_preview(14377)

● Query


,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags,new_songs
14377,[],152634,취향존중 걸그룹 노래 모음,"[400079, 163289, 561640, 632124, 225510, 68085...",22,2018-01-09 18:43:56.000,[],"[400079, 163289, 561640, 632124, 225510, 68085..."



● Recommended
['기분전환', '드라이브', '댄스', '신나는', '여름', '아이돌', '발라드', '추억', '힐링', '설렘']


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
56111,"[GN2503, GN2501, GN2506, GN0201]",20120703,DAY BY DAY,2133148,[412000],DAY BY DAY,"[GN2500, GN0200]",[티아라],56111,"[댄스, 기분전환, 매장음악, 추억, 숨은명곡, 수록곡, 아이돌, 운동, 드라이브,..."
202915,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20120103,Funky Town,2058067,[412000],Lovey-Dovey,"[GN2500, GN0200]",[티아라],202915,"[발라드, 기분전환, 매장음악, 드라이브, 설렘, 사랑, 흥폭발, 댄스, 걸그룹, ..."
265348,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20111019,`The Boys` The 3rd Album,2017426,[228069],The Boys,"[GN2500, GN0200]",[소녀시대 (GIRLS` GENERATION)],265348,"[댄스, 추억, 발라드, 운동, 아이돌, 드라이브, 신나는노래, 운전, 기분전환, ..."
488371,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20120509,UNE ANNEE,2123373,[534360],HUSH,"[GN2500, GN0200]",[Apink (에이핑크)],488371,"[기분전환, 설렘, 사랑, 매장음악, 발라드, 장르불문, 댄스, 드라이브, 댄스곡,..."
684134,"[GN0205, GN0201]",20120113,The Grasshoppers,2064311,[229494],베짱이 찬가,[GN0200],[써니힐],684134,"[발라드, 힙합, 랩, 여름, 신남, 기분전환, 매장음악, 추억, 드라이브, 락, ..."


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
193710,"[GN0105, GN2505, GN2501, GN0101, GN2503]",20131121,그리워해요,2217249,[406712],그리워해요,"[GN2500, GN0100]",[2NE1],193710,"[힙합, 발라드, 댄스, 플레이리스트, 팝송, 팝, 감성, 작업, 슬픔, 이별, 추..."
251375,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20150303,Hi~,2306957,[789331],안녕 (Hi~),"[GN2500, GN0200]",[러블리즈],251375,"[봄, 벚꽃, 힐링, 기분전환, 설렘, 수록곡, 행복, 사랑, 흥폭발, 댄스, 드라..."
318779,"[GN2503, GN2501, GN2506, GN0201]",20150317,The 1st Mini Album `Ice Cream Cake`,2309156,[780066],Take It Slow,"[GN2500, GN0200]",[Red Velvet (레드벨벳)],318779,"[힐링, 휴식, 기분전환, 운동, 설렘, 사랑, 노래, 봄, 아이돌, 잔잔한, 드라..."
455945,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20140804,The 1st Single `행복 (Happiness)`,2270252,[780066],행복 (Happiness),"[GN2500, GN0200]",[Red Velvet (레드벨벳)],455945,"[리듬, 댄스, 산책, 아이돌, 운동, 기분전환, 매장음악, 운동할때, 신나는노래,..."
506824,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20151207,Lovelinus,2654998,[789331],그대에게,"[GN2500, GN0200]",[러블리즈],506824,"[여름, 댄스, 걸그룹, 좋은노래, 신나는, 흥폭발, 띵곡, 아이돌, 응원, 휴식,..."


In [88]:
recommendation_preview(34)

● Query


,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags,new_songs
34,[],148379,카페에 틀어놓기 좋은 연주곡들!,"[161783, 133188, 171517, 640198, 538506, 46043...",12,2018-03-09 15:40:28.000,[],"[85691, 552747, 118675, 20303, 204138, 42323, ..."



● Recommended
['재즈', 'Jazz', '휴식', '카페', '힐링', '커피', '기분전환', '잔잔한', '매장음악', '감성']


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
91049,"[GN1701, GN1704]",20160406,Like Minds,2677966,"[29327, 27205, 30473, 30306, 12928]",For A Thousand Years (Album Ver.),[GN1700],"[Gary Burton, Chick Corea, Roy Haynes, Dave Ho...",91049,"[매장음악, 까페, 가을밤, 가을, Jazz, 재즈, 독서, 생각, 연주, 피아노,..."
149501,[GN1701],20070908,Giovanni Mirabassi & Andrzej Jagodzinski Trio,358792,[229786],El Pueblo Unido Jamas Sera Vencido,[GN1700],[Giovanni Mirabassi & Andrzej Jagodzinski Trio],149501,"[힐링, 휴식, 까페, 밤, 새벽, 연주, 재즈, 비오는날, 감성, 연주곡, Jaz..."
345979,[GN1701],20070711,Lullaby Of Birdland,353787,[227602],Lullaby Of Birdland,[GN1700],[Kenny Drew Trio],345979,"[커피, 피아노, 뉴에이지, 재즈, 연주, 매장음악, 까페, 카페, 겨울]"
493970,"[GN1701, GN1704]",19990309,The Real Mccoy (RVG Edition),317839,[27140],Four By Five,[GN1700],[McCoy Tyner],493970,"[재즈, Jazz, 매장음악, 까페]"
568599,"[GN1701, GN1703]",19990420,The Song Lives On,378490,[20253],Living In Blue,[GN1700],[Joe Sample],568599,"[재즈, 가을감성, 야경, 카페, 오후, 피아노, 가을밤, 가을, Jazz, 독서,..."


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
80898,"[GN1701, GN1704]",20100503,54,878853,"[29775, 16638, 106286]",Imaginary Time,[GN1700],"[John Scofield, Vince Mendoza, Metropole Orche...",80898,[재즈]
113663,"[GN1701, GN1704]",20100503,54,878853,"[29775, 16638, 106286]",Jung Parade,[GN1700],"[John Scofield, Vince Mendoza, Metropole Orche...",113663,[재즈]
380566,"[GN1701, GN1703]",20150925,Past Present,2642160,[29775],Past Present,[GN1700],[John Scofield],380566,"[재즈, Jazz, 팝]"
571887,"[GN1701, GN1704]",20100503,54,878853,"[29775, 16638, 106286]",Honest I Do,[GN1700],"[John Scofield, Vince Mendoza, Metropole Orche...",571887,[재즈]
640264,[GN1701],20020509,Uberjam,65861,[29775],Ideofunk,[GN1700],[John Scofield],640264,"[겨울, 소울, 재즈, Jazz]"


In [89]:
recommendation_preview(21)

● Query


,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags,new_songs
21,[],5611,여름엔 이열치열! 더울땐 발라드 곡이지!,"[214605, 668763, 197752, 165412, 194435, 25627...",8,2016-08-05 19:29:33.000,[],"[214605, 668763, 197752, 165412, 194435, 25627..."



● Recommended
['가을', '휴식', '슬픔', '잔잔한', '이별', '발라드', '힐링', '밤', '새벽', '기분전환']


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
105666,"[GN0105, GN0101]",20141007,내 마음이 뭐가 돼,2284571,[203912],내 마음이 뭐가 돼,[GN0100],[윤하 (YOUNHA)],105666,"[가을밤, 가을, 카페, 가을비, 여행, 바람, 휴식, 산책, 슬픔, 잔잔한, 이별..."
284710,[GN0101],20150209,어차피 한번은 아파야 해,2303732,[658092],어차피 한번은 아파야 해,[GN0100],[유성은],284710,"[힐링, 휴식, 슬픔, 추억, 이별, 회상, 새벽, 감성, 발라드, 밤, 잔잔한, ..."
289496,"[GN0105, GN0101]",20151008,미안해요,2643513,[499464],미안해요,[GN0100],[박보람],289496,"[조용한, 밤, 새벽, 짝사랑, 슬픔, 감성, 설렘, 사랑, 이별, 눈물, 기분전환..."
447225,"[GN0105, GN0101]",20131206,Subsonic,2217929,[203912],괜찮다,[GN0100],[윤하 (YOUNHA)],447225,"[잔잔한, 감성, 밤, 새벽, 취향저격, 발라드, 사랑, 비오는날, 팝, 슬픔, 이..."
591968,"[GN0105, GN0101]",20141126,Genuine,2292441,[643551],우리 다시 사랑한다면,[GN0100],[2BIC(투빅)],591968,"[슬픔, 추억, 이별, 회상, 사랑, 설렘, 기분전환, 드라이브, 발라드, 봄, 카..."


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
281936,"[GN0105, GN0101]",20110531,2011 월간 윤종신 June,1293152,"[437, 242988]",말꼬리,[GN0100],"[윤종신, 정준일]",281936,"[비오는날, 감성, 비, 카페, 사랑노래, 드라이브, 센치, 조용한, 인디, 봄, ..."
298962,"[GN0105, GN0101]",20131220,행보 2013 윤종신,2222589,[437],1월부터 6월까지,[GN0100],[윤종신],298962,"[감성, 카페, 오후, 드라이브, 저녁, 기분전환, 사랑, 여름노래, 이별, 비오는..."
366786,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20101007,가을방학,1035872,[437760],가끔 미치도록 네가 안고 싶어질 때가 있어,"[GN0500, GN0800]",[가을방학],366786,"[슬픔, 추억, 이별, 회상, 기분전환, 설렘, 사랑, 어쿠스틱, 밴드, 우울, 위..."
454528,"[GN0105, GN0101]",20101130,"두 번째, 방",1086280,"[177241, 181716]",부디,[GN0100],"[심규선 (Lucia), 에피톤 프로젝트]",454528,"[힐링, 휴식, 밤, 새벽, 발라드, 비오는날, 추억, 회상, 우울, 눈물, 인디,..."
543820,"[GN0105, GN0101]",20140930,The 1st Digital Single `이 소설의 끝을 다시 써보려 해`,2284064,[711476],이 소설의 끝을 다시 써보려 해,[GN0100],[한동근],543820,"[감성, 가을, 새벽, 우울, 발라드, 가요, 사랑, 이별, 눈물, 듣기좋은, 봄,..."


In [90]:
recommendation_preview(7777)

● Query


,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags,new_songs
7777,[],84202,북카페에서 은은하게 들리던 선율 모음,"[220142, 439629, 270602, 19206, 202828, 395256...",16,2018-07-13 16:39:20.000,[],"[220142, 439629, 270602, 19206, 395256, 130680..."



● Recommended
['잔잔한', '뉴에이지', '힐링', '카페', '휴식', '연주곡', '피아노', '밤', '감성', '산책']


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
102816,"[GN1806, GN1801]",20100413,Le Petit Piano,857554,[1965],비밀,[GN1800],[정재형],102816,"[힐링, 휴식, 밤, 새벽, 감성, 혼자, 우울, 뉴에이지, 마음, 생각, 느낌, ..."
362771,[GN1801],20121212,생각할수록 좋아지는 사람,2172513,[405580],생각할수록 좋아지는 사람,[GN1800],[피아노 포엠],362771,"[스트레스, 밤, 새벽, 카페, 잔잔한, 뉴에이지, 겨울, 감성, 연주곡, 야경, ..."
438092,"[GN1806, GN1801]",19990900,보내지 못한 편지,8332,[1379],설레임,[GN1800],[김광민],438092,"[힐링, 휴식, 밤, 새벽, 모닝콜, 여유, 가을, 잔잔한, 뉴에이지, 겨울, 연주..."
487339,[GN1801],20070314,오늘 하루가 선물입니다.,347554,[48260],바람소리,[GN1800],[메이세컨],487339,"[힐링, 휴식, 배경음악, 카페음악, 매장음악, 밤, 연주곡, 새벽, 새벽감성, 뉴..."
552922,"[GN1805, GN1803, GN1801]",20030300,A Magic Wand Of Standards,30390,[103644],Sunflower,[GN1800],[Gontiti],552922,"[피아노, 카페, 잔잔한, 추억, 위로, 힐링, 조용한, 뉴에이지, 자장가, 숙면,..."


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
22129,"[GN1802, GN1801]",20120615,Dentist Hongilkim 01,2126808,[11048],Thank You,[GN1800],[라팡 (lapin)],22129,"[힐링, 커피, 매장, 피아노, 카페, 연주곡, 잔잔한, 독서, 뉴에이지, 자장가,..."
79010,[GN1701],20040310,Dancing Queen,34125,[101502],Time After Time,[GN1700],[European Jazz Trio],79010,"[집중, 노동요, 연주곡, 주말, 피아노, 봄날, 산책, 자장가, 공부, 따뜻한, ..."
80738,[GN1801],20140321,바람처럼 스쳐 간 사랑,2243684,[736217],바람처럼 스쳐 간 사랑,[GN1800],[Memorize],80738,"[집중, 휴식, 잔잔한, 새벽, 독서, 힐링, 공부, 따뜻한, 뉴에이지, 비오는날, 밤]"
562636,"[GN1806, GN1801]",20160822,피아노 지브리 (Piano Ghibli) (Korea Tour Edition),2705626,[450121],벼랑위의 포뇨,[GN1800],[Elizabeth Bright],562636,"[여유, 카페, 오후, 피아노, 연주곡, 독서, 커피, 집중, 클래식, 뉴에이지, ..."
588039,[GN1701],20110624,Prismatic,1317451,[441362],남촌 (Piano Ver.),[GN1700],[김가온],588039,"[집중, 노동요, 연주곡, 주말, 피아노, 봄날, 산책, 자장가, 공부, 따뜻한, ..."


In [91]:
recommendation_preview(160)

● Query


,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags,new_songs
160,"[카페, 펍, 그루브한]",106776,SNS에 올리고픈 그 감성! 그루비 BGM!,"[647304, 128806, 130968, 682476, 698813, 17765...",20,2018-05-22 12:17:36.000,"[카페, 펍]","[647304, 128806, 130968, 682476, 698813, 17765..."



● Recommended
['감성', '잔잔한', '기분전환', '드라이브', '새벽', '그루브', '팝', '휴식', 'RnB', '트렌디']


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
100702,"[GN0401, GN0403, GN0402]",20160506,Interlude,2683503,[674710],Castaway (Feat. MISO),[GN0400],[Crush],100702,"[밤, 까페, 새벽, 라운지, 매장음악, EDM, 일렉트로니카, 내적댄스, 몽환적,..."
483849,"[GN2601, GN2603, GN2602]",20160627,You,2693836,[990473],See From The Sea (Feat. YESEO),[GN2600],[Deepshower (딥샤워)],483849,"[라운지, 매장음악, EDM, 일렉트로니카, 내적댄스, 몽환적, 편집샵, 힙합, 감..."
547342,"[GN2601, GN0509, GN2603, GN0504, GN0501]",20160922,High,10000248,[1183362],High (Feat. Rachel Lim),"[GN0500, GN2600]",[JIDA (지다)],547342,"[겨울, 감성, 발라드, 추억, 새해, 기분전환, 사랑, 까페, 싱어송라이터, 인디..."
559870,"[GN0401, GN0403]",20151026,EMOTION,2646544,"[907217, 623812]",EMOTION,[GN0400],"[dress, Lydia Paek]",559870,"[감성, 느낌있는, 밤, 카페, 새벽, 분위기, 알앤비, 사랑, 이별, RnB, 음..."
653392,"[GN2601, GN0401, GN2603, GN0402]",20161109,photograph,10013499,[1183062],Photograph,"[GN0400, GN2600]",[offonoff (오프온오프)],653392,"[힙합, 리듬, 국내힙합, HipHop, 비트, 여름, 드라이브, 바다, 휴가, 감..."


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
116354,"[GN1304, GN1301, GN1308]",20180504,Cold Fire,10164069,[1958188],Cold Fire (Feat. DEAN),[GN1300],[PREP],116354,"[밤, 감각적인, 분위기, 기분전환, 혼술, 세련된, 여름, 페스티벌, EDM, 힙..."
226191,"[GN0908, GN0901]",20181002,Nina,10208464,[1627730],Nina,[GN0900],[Daniel March],226191,"[가을감성, 감성, 매장음악, 해외, 가을, 알앤비, 음색, 기분전환, 팝, 소울,..."
257288,[GN1301],20180607,Shy,10173349,"[792656, 166141]",Shy,[GN1300],"[Tuxedo, Zapp]",257288,"[주말, 댄스, Pop, 기분전환, 파티, 내적댄스, 스트레스, 신나는, 매장음악,..."
506176,"[GN1102, GN1101]",20160208,ピンクのダンス / Pink Dance EP,2670401,[956844],Crazy Nights 狂氣夜 / Crazy Nights Kyoukiyo (Craz...,[GN1100],[Future Girlfriend],506176,"[감성, 여름밤, 드라이브, 그루브, 한강, 기분전환, 시원한, 여름, 하우스, 일..."
599616,"[GN1912, GN1902, GN1901]",20170621,何度でも新しく生まれる / Nandodemo Atarasiku Umareru (몇번이...,10072874,[2846],ラビリンス / Labyrinth (Vocal : 滿島ひかり),[GN1900],[Mondo Grosso],599616,"[감성, 카페, 주말, 인디팝, 그루브, 오후, 어쿠스틱, JPOP, 해외, RnB..."


In [92]:
recommendation_preview(0) # 참고할 수 있는 정보가 없어서 인기 곡, 인기 태그로 채워진 경우

● Query


,tags,id,plylst_title,songs,like_cnt,updt_date,new_tags,new_songs
0,"[Billboard, uspop]",147332,랜덤 플레이 뮤직 - 팝,[],4,2020-03-31 00:45:30.000,[],[]



● Recommended
['발라드', '설렘', '사랑', '잔잔한', '감성', '기분전환', '비오는날', '카페', '드라이브', '밤']


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
116573,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20111123,Lo9ve3r4s,2038488,[242988],안아줘,"[GN0500, GN0600]",[정준일],116573,"[슬픔, 밤, 이별, 새벽, 감성, 잔잔한, 추억, 출근길, 사랑, 발라드, 겨울밤..."
144663,"[GN0105, GN0101]",20170324,밤편지,10047890,[261143],밤편지,[GN0100],[아이유],144663,"[힙합, 발라드, 댄스, 플레이리스트, 팝송, 팝, 까페, 설렘, 사랑, 잔잔한, ..."
357367,"[GN0401, GN0403]",20160621,비,2692501,[752425],비,[GN0400],[폴킴],357367,"[비오는날, 감성, 비, 카페, 사랑노래, 드라이브, 센치, 조용한, 인디, 기분좋..."
366786,"[GN0805, GN0509, GN0502, GN0801, GN0501]",20101007,가을방학,1035872,[437760],가끔 미치도록 네가 안고 싶어질 때가 있어,"[GN0500, GN0800]",[가을방학],366786,"[슬픔, 추억, 이별, 회상, 기분전환, 설렘, 사랑, 어쿠스틱, 밴드, 우울, 위..."
654757,"[GN1501, GN1504]",20041115,미안하다 사랑한다 OST,43841,[1191],눈의 꽃,[GN1500],[박효신],654757,"[휴식, 힐링, 여행, 산책, 감성, 밤, 발라드, 추억, 드라이브, 2000년대,..."


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,new_tags
6546,"[GN0105, GN1501, GN0101, GN1504]",20140902,연애의 발견 OST Part 4,2279896,[192827],"묘해, 너와","[GN1500, GN0100]",[어쿠스틱 콜라보],6546,"[기분전환, 매장음악, 까페, 설렘, 사랑, 겨울, 감성, 발라드, 눈, 잔잔, 이..."
88503,"[GN0104, GN1501, GN0101, GN1504]",20050120,쾌걸춘향 OST,48736,[164914],응급실,"[GN1500, GN0100]",[Izi],88503,"[감성, 밤, 발라드, 추억, 드라이브, 2000년대, 사랑, 이별, 드라마, 명곡..."
443914,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20150723,여자친구 2nd Mini Album `Flower Bud`,2330981,[828478],오늘부터 우리는 (Me Gustas Tu),"[GN2500, GN0200]",[여자친구 (GFRIEND)],443914,"[댄스, 여름, 걸그룹, 좋은노래, 신나는, 설렘, 사랑, 봄, 기분전환, 매장음악..."
459256,[GN0901],20141110,Uptown Funk (Feat. Bruno Mars),2290446,[45077],Uptown Funk (Feat. Bruno Mars),[GN0900],[Mark Ronson],459256,"[휴식, 힐링, 잔잔한, 기분전환, 매장음악, 여행, 까페, 산책, 스트레스, 드라..."
640657,"[GN0105, GN0101]",20160905,내가 말하고 싶은건,2708791,[523900],내가 말하고 싶은건,[GN0100],[김제훈],640657,"[감성, 밤, 카페, 봄노래, 인디팝, 드라이브, 봄날, 밤새벽, 사랑, 커피숍, ..."


# 🌵 **추천 결과 저장**

In [150]:
answers = []
state = 0

for d in val_dict:
    id, new_songs, new_tags, songs, tags = d.values()

    # 1 ) 참고할 노래가 없는 경우
    if len(new_songs) == 0:
         # 1. 노래는 없지만 tag가 있다면 tag를 기반으로 추천해준다.
        if new_tags:
            rec_songs = tag_based_song_recommend(new_tags, len(new_tags))
            rec_songs = fill(rec_songs) # 100곡을 채우지 못한 경우 인기곡으로
            rec_tags = tag_recommend(rec_songs, tags)
        else:
            # 2. 이용할 정보가 없는 경우 인기곡, 인기 태그를 추천한다.
            rec_songs = pop_songs
            rec_tags = pop_tags2

    # 2 ) 참고할 노래가 있는 경우 유사곡 k개(모두 골고루 가질 수 있도록 배치했음)씩을 모아 100곡을 완성
    else:
        rec_songs = similarity_based_song_recommend(new_songs)

        # # 100곡을 채우지 못한 경우 tag를 이용해 나머지를 채워준다.
        # if len(rec_songs) < 100:
        #     if new_tags:
        #         for song in tag_based_song_recommend(new_tags, len(new_tags), 30):
        #             if len(rec_songs) == 100:
        #                 break
        #             if song not in songs and song not in rec_songs:
        #                 rec_songs.append(song)

        # tag가 없어 채우지 못한 경우 인기곡으로 채워준다.
        rec_songs = fill(rec_songs)
        rec_tags = tag_recommend(rec_songs, tags)
        
    assert len(set(rec_songs)) == 100

    if state%1000 == 0:
        print(state,"하는즁입니돠")
    state += 1

    answers.append({'id':id,
        'songs':rec_songs,
        'tags':rec_tags})

0 하는즁입니돠
1000 하는즁입니돠
2000 하는즁입니돠
3000 하는즁입니돠
4000 하는즁입니돠
5000 하는즁입니돠
6000 하는즁입니돠
7000 하는즁입니돠
8000 하는즁입니돠
9000 하는즁입니돠
10000 하는즁입니돠
11000 하는즁입니돠
12000 하는즁입니돠
13000 하는즁입니돠
14000 하는즁입니돠
15000 하는즁입니돠
16000 하는즁입니돠
17000 하는즁입니돠
18000 하는즁입니돠
19000 하는즁입니돠
20000 하는즁입니돠
21000 하는즁입니돠
22000 하는즁입니돠
23000 하는즁입니돠


In [151]:
len(answers)

23015

In [152]:
for d in answers:
    id, songs, tags = d.values()
    if len(set(songs)) != 100 or len(set(tags)) != 10:
        print(id,len(set(songs)), len(set(tags)))
        break

In [153]:
import io
import json

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [154]:
write_json(answers,"new_AE_1110_tags2.json")